- All non-negative coefficients
- Upper bounded Intercept and the feature with expected lowest coefficient
- Bounds for Intercept depend on no_intercept flag also
- Specific order for some coefficients
- Min percentage by which the coefficients (of ordered features) differ

# Imports

In [5]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from scipy.optimize import lsq_linear

# User inputs

In [6]:
df = pd.read_csv("data/train.csv")

In [7]:
no_intercept = True

In [8]:
target = 'SalePrice'

In [9]:
# Features in the expected ascending order of coefficients
features_ordered = ['PoolArea', 'LotArea', 'TotalBsmtSF', 'GarageArea']
# Features without any order for coefficients
features_unordered = ['GrLivArea']
features = features_ordered + features_unordered
features

['PoolArea', 'LotArea', 'TotalBsmtSF', 'GarageArea', 'GrLivArea']

In [10]:
# Min percentage gaps between successive (ordered) features
min_gap_pct = [None, 0.1, 0.05, 0]

# Constraints

In [11]:
# Initialize coefficients
len_coeffs = len(features) + 1
coeffs = list(np.zeros(len_coeffs))
print("Initialized coefficients:", coeffs)

Initialized coefficients: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [19]:
# Put constraints 
min_con = list(np.zeros(len_coeffs))

max_con = [10, 1, np.inf, np.inf, np.inf, np.inf]
if no_intercept:
    max_con[0] = 0.0001

print("Minimum constraints:", min_con)
print("Maximum constraints:", max_con)

Minimum constraints: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Maximum constraints: [0.0001, 1, inf, inf, inf, inf]


# Model

y = X0 + X1*PoolArea + (1.1*X1+X2)*LotArea + (1.05*(1.1*X1+X2)+X3)*TotalBsmtSF + (1.05*(1.1*X1+X2)+X3+X4)*GarageArea + X5*GrLivArea

y = X0 + X1*(PoolArea+1.1*LotArea+1.05*1.1*TotalBsmtSF+1.05*1.1*GarageArea) + X2*(LotArea+1.05*TotalBsmtSF+1.05*GarageArea) + X3*(TotalBsmtSF+GarageArea) + X4*GarageArea + X5*GrLivArea

In [12]:
# Feature engineer   
X = df[features].copy()
X['F1'] = X[features[0]] + (1+min_gap_pct[1])*X[features[1]] + (1+min_gap_pct[2])*(1+min_gap_pct[1])*X[features[2]] + (1+min_gap_pct[2])*(1+min_gap_pct[1])*X[features[3]]
X['F2'] = X[features[1]] + (1+min_gap_pct[2])*X[features[2]] + (1+min_gap_pct[2])*X[features[3]]
X['F3'] = X[features[2]] + X[features[3]]
X['F4'] = X[features[3]]
X['F5'] = X[features[4]]
X = X.drop(features, axis=1)

In [13]:
# Convert independent variables to a matrix
X = X.values

# Add an array of ones to act as intercept coefficient
ones = np.ones(X.shape[0])
# Combine array of ones and indepedent variables
X = np.concatenate((ones[:, np.newaxis], X), axis=1)
X

array([[1.000000e+00, 1.091662e+04, 9.924200e+03, 1.404000e+03,
        5.480000e+02, 1.710000e+03],
       [1.000000e+00, 1.254891e+04, 1.140810e+04, 1.722000e+03,
        4.600000e+02, 1.262000e+03],
       [1.000000e+00, 1.413984e+04, 1.285440e+04, 1.528000e+03,
        6.080000e+02, 1.786000e+03],
       ...,
       [1.000000e+00, 1.156782e+04, 1.051620e+04, 1.404000e+03,
        2.520000e+02, 2.340000e+03],
       [1.000000e+00, 1.221099e+04, 1.110090e+04, 1.318000e+03,
        2.400000e+02, 1.078000e+03],
       [1.000000e+00, 1.270016e+04, 1.154560e+04, 1.532000e+03,
        2.760000e+02, 1.256000e+03]])

In [14]:
# Convert target variable to a matrix
y = df[target].values
y

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [20]:
# Run optimization
results = lsq_linear(X, y, bounds=(min_con, max_con), lsmr_tol='auto')
print("Results:\n", results)

Results:
  active_mask: array([-1, -1,  0,  0,  0,  0])
        cost: 1591264994306.908
         fun: array([-15570.8800255 ,  -6211.14423445, -17020.4392965 , ...,
       -51622.04361478,  -6813.20555247,   9554.21332259])
     message: 'The relative change of the cost function is less than `tol`.'
         nit: 16
  optimality: 0.000710548149206837
      status: 2
     success: True
           x: array([2.77598257e-36, 6.57413066e-15, 1.81752451e-01, 4.25750706e+01,
       5.33262444e+01, 5.97235036e+01])


In [21]:
if results.success:
    # Transform the coefficients back to the context of original features 
    coeffs[0] = results.x[0]
    coeffs[1] = results.x[1]
    coeffs[2] = (1+min_gap_pct[1])*results.x[1] + results.x[2]
    coeffs[3] = (1+min_gap_pct[2])*((1+min_gap_pct[1])*results.x[1] + results.x[2]) + results.x[3]
    coeffs[4] = (1+min_gap_pct[2])*((1+min_gap_pct[1])*results.x[1] + results.x[2]) + results.x[3] + results.x[4]
    coeffs[5] = results.x[5]
    print("Final Coefficients (including intercept):", coeffs)
else:
    print("Convergence was not achieved!")

Final Coefficients (including intercept): [2.77598257138416e-36, 6.5741306644302445e-15, 0.1817524511117668, 42.765910708107825, 96.09215507019684, 59.72350363625727]
